# Check Data Quality of the Files

checking data quality of the files
e.g. if there are redundant files or empty files, etc.

Steps:
- importing Boto because files are on S3

In [1]:
#!pip3 install s3fs boto3 mock --use-feature=2020-resolver
# there is a bug in the feature resolver, that the installation runs forever, even though the installation succeeded.
# you may have to interrupt the kernel and then s3fs is isntalled properly

^C


In [5]:
import boto3
import pandas as pd

In [16]:
def file_list(bucket_name):
    """A function that lists all files (objects) in a given bucket (argument bucket_name) and puts it in a list"""
    s3 = boto3.resource('s3')
    my_bucket = s3.Bucket(bucket_name)
    return [i.key for i in my_bucket.objects.all()]

In [17]:
file_list('dsti-ml-nlp-class')

['Data/MR_IBRAHIM_ASHAFA_STATEMENT (1).csv',
 'Data/MR_IBRAHIM_ASHAFA_STATEMENT (2).csv',
 'Data/MR_IBRAHIM_ASHAFA_STATEMENT (3).csv',
 'Data/MR_IBRAHIM_ASHAFA_STATEMENT (4).csv',
 'Data/MR_IBRAHIM_ASHAFA_STATEMENT.csv',
 'Data/MR_TAPIYA_JOHN_BANK_STATEMENT-part2.csv',
 'Data/Mr_Tochukwu_statement-part2.csv',
 'Data/mr_gani_olalekan_statement_July_9.xlsx']

In [3]:
def file_to_pandas(s3_file_uri):
    data = pd.read_csv(s3_file_uri)
    return data

In [6]:
file_to_pandas('s3://dsti-ml-nlp-class/Data/MR_IBRAHIM_ASHAFA_STATEMENT (1).csv')

,Trans. Date,Reference,Value. Date,Debits,Credits,Balance,Remarks_processed,ACCOUNT_TYPE,BANK_ID,FILE_NAME,filtered_description,processed_description,CLASSE
0,07-Dec-2020,'0GAPS,07-Dec-2020,NaN,"308,750.00","278,851.80",TRANSFER BETWEEN CUSTOMERS via GAPScom on Baba...,SAVINGS ACCOUNT,GT_Bank,./static/raw/MR_IBRAHIM_ASHAFA_STATEMENT.pdf,TRANSFER BETWEEN CUSTOMERS via GAPScom on Baba...,TRANSFER BETWEEN CUSTOMERS via GAPScom on Baba...,transfert
1,07-Dec-2020,'0GAPS,07-Dec-2020,NaN,"82,092.39","360,944.19",TRANSFER BETWEEN CUSTOMERS via GAPScom on Eken...,SAVINGS ACCOUNT,GT_Bank,./static/raw/MR_IBRAHIM_ASHAFA_STATEMENT.pdf,TRANSFER BETWEEN CUSTOMERS via GAPScom on Eken...,TRANSFER BETWEEN CUSTOMERS via GAPScom on Eken...,transfert
2,07-Dec-2020,'0GWTR,07-Dec-2020.1,"50,000.00",NaN,"310,944.19",TRANSFER BETWEEN CUSTOMERS Via GTWorld Eneke ...,SAVINGS ACCOUNT,GT_Bank,./static/raw/MR_IBRAHIM_ASHAFA_STATEMENT.pdf,TRANSFER BETWEEN CUSTOMERS Via GTWorld Eneke R...,TRANSFER BETWEEN CUSTOMERS Via GTWorld Eneke R...,transfert
3,07-Dec-2020,'0NIPG,07-Dec-2020,"140,000.00",NaN,"170,944.19",NIBSS Instant Payment Outward 0000132012070917...,SAVINGS ACCOUNT,GT_Bank,./static/raw/MR_IBRAHIM_ASHAFA_STATEMENT.pdf,NIBSS Instant Payment Outward via GTWORLD Baba...,NIBSS Instant Payment Outward via GTWORLD Baba...,transfert
4,07-Dec-2020,'0NIPG,07-Dec-2020,50.00,NaN,"170,894.19",COMMISSION 000013201207091758000279938909 NIP ...,SAVINGS ACCOUNT,GT_Bank,./static/raw/MR_IBRAHIM_ASHAFA_STATEMENT.pdf,COMMISSION NIP TRANSFER COMMISSION FOR via GTW...,COMMISSION NIP TRANSFER COMMISSION FOR via GTW...,commission
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,09-Jun-2021,'0USAT,09-Jun-2021,"1,000.00",NaN,"9,127.60",Airtime Purchase GTWORLD_QL- 101CT000000000265...,SAVINGS ACCOUNT,GT_Bank,./static/raw/MR_IBRAHIM_ASHAFA_STATEMENT.pdf,Airtime Purchase GTWORLD_QL CT,Airtime Purchase GTWORLDQL CT,utility
1279,09-Jun-2021,'0NIPG,09-Jun-2021,"4,000.00",NaN,"5,127.60",NIBSS Instant Payment Outward 0000132106091420...,SAVINGS ACCOUNT,GT_Bank,./static/raw/MR_IBRAHIM_ASHAFA_STATEMENT.pdf,NIBSS Instant Payment Outward via GTWORLD TO Y...,NIBSS Instant Payment Outward via GTWORLD TO Y...,transfert
1280,09-Jun-2021,'0NIPG,09-Jun-2021,10.00,NaN,"5,117.60",COMMISSION 000013210609142049000172467995 NIP ...,SAVINGS ACCOUNT,GT_Bank,./static/raw/MR_IBRAHIM_ASHAFA_STATEMENT.pdf,COMMISSION NIP TRANSFER COMMISSION FOR via GTW...,COMMISSION NIP TRANSFER COMMISSION FOR via GTW...,commission
1281,09-Jun-2021,'0NIPG,09-Jun-2021,.75,NaN,"5,116.85",VALUE ADDED TAX 000013210609142049000172467995...,SAVINGS ACCOUNT,GT_Bank,./static/raw/MR_IBRAHIM_ASHAFA_STATEMENT.pdf,VALUE ADDED TAX VAT ON NIP TRANSFER FOR via GT...,VALUE ADDED TAX VAT ON NIP TRANSFER FOR via GT...,tax
